### Imports

In [1]:
from langchain_community.graphs import Neo4jGraph
from langchain_openai import ChatOpenAI
from langchain.chains import GraphCypherQAChain

### Connect to the Neo4j database and instantiate an LLM Transformer

In [2]:
openai_apikey = ""
url = "bolt://localhost:7687"
username = "neo4j"
password = ""
graph = Neo4jGraph(url=url, username=username, password=password)

llm = ChatOpenAI(temperature=0, model_name="gpt-4-0125-preview", openai_api_key=openai_apikey)

### Query the Graph using GraphCypherQAChain

In [3]:
chain = GraphCypherQAChain.from_llm(graph=graph, llm=llm, verbose=True, validate_cypher=True)
response = chain.invoke({"query": "Who are the people attending the kickoff meeting?"})

print(response)



> Entering new GraphCypherQAChain chain...
Generated Cypher:
cypher
MATCH (:Event {id: "kickoff"})<-[:THANKED_FOR]-(:Person)-[:THANKED]->(group:Group)
RETURN DISTINCT group.id AS GroupID

Full Context:
[]

> Finished chain.
{'query': 'Who are the people attending the kickoff meeting?', 'result': "I don't know the answer."}


### Observations

I noticed that unless the words in the query exactly matched the entities and relationships in the graph, it did not return an answer.

In this case, even if it did match an answer was not returned.

In [4]:
chain = GraphCypherQAChain.from_llm(graph=graph, llm=llm, verbose=True, validate_cypher=True)
response = chain.invoke({"query": "Who is organizing the Kickoff Meeting?"})

print(response)



> Entering new GraphCypherQAChain chain...
Generated Cypher:
cypher
MATCH (:Event {id: "Kickoff Meeting"})<-[:THANKED_FOR]-(:Person)-[:THANKED]->(group:Group)
RETURN group.id AS Organizer

Full Context:
[]

> Finished chain.
{'query': 'Who is organizing the Kickoff Meeting?', 'result': "I don't know the answer."}
